In [3]:
import os
import datetime
import timedelta
import pandas as pd
import urllib.request, json 

date = (datetime.datetime.now() - timedelta.Timedelta(days=0)).date()
# present = datetime.datetime.now().date()

folders = ["INPUT","RAWCSV","LOG"]
for folder in folders:
    if not os.path.isdir(os.path.join("..","..",folder,str(date))):
        os.mkdir(os.path.join("..","..",folder,str(date)))
        print("Created")

In [4]:
source = pd.read_csv("../../sources.csv")

In [5]:
def getPreviousRawFile(date,state):
    prevDate = date - timedelta.Timedelta(days=1)
    try:
        state_df = pd.read_csv("../../RAWCSV/"+str(prevDate)+"/"+state+"_raw.csv")
    except:
        state_df = getPreviousRawFile(prevDate,state)
    return state_df

In [6]:
path = "../../RAWCSV/"+str(date)+"/"
dir_list = os.listdir(path)
for state in source["StateCode"]:
    if state+"_raw.csv" not in dir_list:
        df = getPreviousRawFile(date,state)
        df["Date"] = str(date)
        df.to_csv(path+state+"_raw.csv",index=False)

In [5]:
df_intTT = pd.read_csv(path+"TT_raw.csv")

In [6]:
df_intTT["Date"] = str(date)
df_intTT["tested_source_state"] = "https://www.icmr.gov.in/"

In [7]:
value = ["Confirmed","Deceased","Recovered","Tested"]
for idx in df_intTT.index:
    stateCode = source[source['StateName'] == df_intTT["District"][idx]]["StateCode"].item()
    df_state = pd.read_csv(path + stateCode + "_raw.csv")
    for val in value:
        df_intTT["cumulative{}NumberForDistrict".format(val)][idx] = df_state["cumulative{}NumberForState".format(val)][0]
        
for val in value:
    df_intTT["cumulative{}NumberForState".format(val)] = df_intTT["cumulative{}NumberForDistrict".format(val)].sum()

<ipython-input-7-653a08d311f0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_intTT["cumulative{}NumberForDistrict".format(val)][idx] = df_state["cumulative{}NumberForState".format(val)][0]


In [8]:
def addVaccination(df,date):
    # df = df.drop(['cumulativeVaccinatedNumberForDistrict', 'cumulativeVaccinated1NumberForDistrict', 'cumulativeVaccinated2NumberForDistrict'], axis=1)
    with urllib.request.urlopen("https://api.cowin.gov.in/api/v1/reports/v2/getPublicReports?date="+str(date)) as url:
        data = json.loads(url.read().decode())
    df["cumulativeVaccinated1NumberForState"] = data["topBlock"]["vaccination"]["tot_dose_1"]
    df["cumulativeVaccinated2NumberForState"] = data["topBlock"]["vaccination"]["tot_dose_2"]
    df["cumulativeVaccinatedNumberForState"] = data["topBlock"]["vaccination"]["total_doses"]
    states = []
    vaccinated = []
    vaccinated1 = []
    vaccinated2 = []
    for sta in data['getBeneficiariesGroupBy']:
        if (sta['title'].lower() == "dadra and nagar haveli") or (sta['title'].lower() == "daman and diu"):
            states.append('Dadra and Nagar Haveli and Daman and Diu')
        else:
            states.append(sta['title'])
        vaccinated.append(sta['total'])
        vaccinated1.append(sta['partial_vaccinated'])
        vaccinated2.append(sta['totally_vaccinated'])
    VAC_df = pd.DataFrame(list(zip(states,vaccinated,vaccinated1,vaccinated2))).groupby(0, as_index=False).sum()
    VAC_df = VAC_df.rename(columns={0:"District",1:"cumulativeVaccinatedNumberForDistrict",2:"cumulativeVaccinated1NumberForDistrict",3:"cumulativeVaccinated2NumberForDistrict",})
    df = df.merge(VAC_df,how="left",on="District")
    return df

In [9]:
df_intTT = addVaccination(df_intTT,date)

In [10]:
def getDeltaValues(df,date):
    prevDate = date - timedelta.Timedelta(days=1)
    cs = pd.read_csv("../../RAWCSV/"+str(prevDate)+"/TT_final.csv")
    df["deltaConfirmedForState"] = df["cumulativeConfirmedNumberForState"] - cs["cumulativeConfirmedNumberForState"][0]
    df["deltaDeceasedForState"] = df["cumulativeDeceasedNumberForState"] - cs["cumulativeDeceasedNumberForState"][0]
    df["deltaRecoveredForState"] = df["cumulativeRecoveredNumberForState"] - cs["cumulativeRecoveredNumberForState"][0]
    df["deltaTestedForState"] = df["cumulativeTestedNumberForState"] - cs["cumulativeTestedNumberForState"][0]
    return df

In [11]:
df_intTT = getDeltaValues(df_intTT,date)

In [13]:
df_intTT.head()

,Unnamed: 0,Date,State/UTCode,District,tested_last_updated_district,tested_source_district,notesForDistrict,cumulativeConfirmedNumberForDistrict,cumulativeDeceasedNumberForDistrict,cumulativeRecoveredNumberForDistrict,...,cumulativeVaccinatedNumberForState,cumulativeVaccinated1NumberForState,cumulativeVaccinated2NumberForState,cumulativeVaccinatedNumberForDistrict_y,cumulativeVaccinated1NumberForDistrict,cumulativeVaccinated2NumberForDistrict,deltaConfirmedForState,deltaDeceasedForState,deltaRecoveredForState,deltaTestedForState
0,0,2022-01-08,TT,Andaman and Nicobar Islands,NaN,NaN,NaN,7651,129,7518,...,1516070455,885326820,630743635,611401,315178,296223,-215322,-975,-98142,4420880.0
1,1,2022-01-08,TT,Andhra Pradesh,NaN,NaN,NaN,2079763,14501,2062290,...,1516070455,885326820,630743635,73191055,42032859,31158196,-215322,-975,-98142,4420880.0
2,2,2022-01-08,TT,Arunachal Pradesh,NaN,NaN,NaN,55436,282,55051,...,1516070455,885326820,630743635,1483344,826902,656442,-215322,-975,-98142,4420880.0
3,3,2022-01-08,TT,Assam,NaN,NaN,NaN,610433,5996,600738,...,1516070455,885326820,630743635,38946052,22609990,16336062,-215322,-975,-98142,4420880.0
4,4,2022-01-08,TT,Bihar,NaN,NaN,NaN,735852,12100,715262,...,1516070455,885326820,630743635,103741560,60230336,43511224,-215322,-975,-98142,4420880.0


In [ ]:
# AP
case1 : AP.raw is present >> run update derived value >> log.json

case2 : No AP.raw >> AP_final(Prev Day) and change date >> Do not run(Updatederived Value)

#TT_int
for all state files:
    for a state > Confirmed >> TT_raw_int(District Level)[Recoved,D,T,v1,v2] 
    
StateTotal == sum(DistrictCount)[C,R,D,T,V1,V2]

(TT_final_int)Delta = TT_raw_int - TT_final(day-1)

7dma(TT_final_int,interim = True)

JSON Creations